## Movie Recommendation Project


In [21]:
# Shreyan Chatterjee
# IITB

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer # TfidfVectorizer - This is used to convert text data into numerical values
from sklearn.metrics.pairwise import cosine_similarity

df=pd.read_csv('C:\\Users\\deeps\\Downloads\\Python Assignments\\Python Projects fo Placement\\Movie Recommendation [Project]\\movies.csv')
df

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [4]:
df.shape

(4803, 24)

In [8]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [9]:
# select features to use for recommendation
selected_features=['genres','keywords','original_language','cast','director']
print(selected_features)

['genres', 'keywords', 'original_language', 'cast', 'director']


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [6]:
# check for missing data
df.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [7]:
# see various info of data
df.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [11]:
# display data in selected feature columns
df[selected_features].head()

,genres,keywords,original_language,cast,director
0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,en,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,en,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Action Adventure Crime,spy based on novel secret agent sequel mi6,en,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,en,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,Action Adventure Science Fiction,based on novel mars medallion space travel pri...,en,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [12]:
# null values in selected columns
df[selected_features].isna().sum()

genres                28
keywords             412
original_language      0
cast                  43
director              30
dtype: int64

In [24]:
# fill null values with null strings
for feature in selected_features:
    df[feature] = df[feature].fillna('')
display (df.head())
display(df[selected_features].isna().sum())

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


genres               0
keywords             0
original_language    0
cast                 0
director             0
dtype: int64

In [31]:
# combine all selected features
combined_features=df['genres']+' '+df['keywords']+' '+df['original_language']+' '+df['cast']+' '+df['director']
display(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  en Edward Burns Kerry Bish\u00...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      en Daniel Henney Eliza Coupe Bill Paxton Ala...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [32]:
# Converting the text data to feature vectors
# This is to find cosine similarity 

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
display (feature_vectors.shape)
print (feature_vectors)


(4803, 14868)

  (0, 2063)	0.18665186499447553
  (0, 6551)	0.12242341251354961
  (0, 11232)	0.209898604739424
  (0, 8840)	0.17353590912089956
  (0, 7505)	0.24540176320809437
  (0, 12584)	0.1637235952387991
  (0, 14312)	0.2144334213116376
  (0, 12150)	0.22256873370821878
  (0, 11506)	0.23530526953263461
  (0, 14843)	0.21826588105197828
  (0, 14582)	0.25549825688355415
  (0, 11533)	0.16260042698137087
  (0, 4168)	0.037343528466907444
  (0, 12316)	0.23530526953263461
  (0, 2758)	0.26972846547717394
  (0, 14256)	0.13883081597626545
  (0, 12403)	0.3766735943238939
  (0, 4893)	0.19824203143913924
  (0, 2618)	0.24259258332105782
  (0, 3119)	0.2311715546144746
  (0, 4573)	0.1121952263281294
  (0, 11761)	0.11206402612776617
  (0, 4442)	0.12012539827085399
  (0, 144)	0.09827307248858286
  (0, 86)	0.08502902364077554
  :	:
  (4801, 14819)	0.322455239802952
  (4801, 4102)	0.2761196091597497
  (4801, 253)	0.19806508189922148
  (4801, 5852)	0.322455239802952
  (4801, 10080)	0.24085559924078767
  (4801, 1384)	0.174

In [33]:
# Getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)
print  (similarity )


[[1.         0.02820855 0.04616569 ... 0.00128325 0.00152922 0.00127856]
 [0.02820855 1.         0.01627439 ... 0.01762654 0.00131869 0.00110253]
 [0.04616569 0.01627439 1.         ... 0.0013247  0.06723507 0.00131986]
 ...
 [0.00128325 0.01762654 0.0013247  ... 1.         0.00140718 0.0266944 ]
 [0.00152922 0.00131869 0.06723507 ... 0.00140718 1.         0.00140204]
 [0.00127856 0.00110253 0.00131986 ... 0.0266944  0.00140204 1.        ]]


In [34]:
# Print Shape Cosine Similarity
display(similarity.shape)


(4803, 4803)

In [130]:
# Enter the movie name to get Similarity 
movie_name = input(' Enter your favourite movie name : ')


 Enter your favourite movie name : ice age


In [131]:
# create list with all movie names given in the dataset

list_titles=df['title'].tolist()
#display(list_titles)
len(list_titles)

4803

In [132]:
# find close matches to the entered movie name by user

find_close_match = difflib.get_close_matches(movie_name, list_titles)
print(find_close_match)

# display closest match
close_match = find_close_match[0]
print(close_match)


['Ice Age', 'Time Changer', 'Office Space']
Ice Age


In [133]:
index_of_movie=df[df['title']==close_match]['index'].values[0]
index_of_movie

631

In [137]:
# Get similarity Row for selected index
# These are the similarity values for the movie entered by the user 

similarity_score = list(enumerate(similarity[index_of_movie]))

# print(similarity_score)
# display(len(similarity_score))

In [138]:
# Sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
#print(sorted_similar_movies)


In [139]:
# Print the name of similar movies based on the index – Top 30 

print('Movies suggested for you:-')

for i in range(0,30):
    index=sorted_similar_movies[i][0]
    print(df[df['index']==index]['title'].values[0])

Movies suggested for you:-
Ice Age
Ice Age: Continental Drift
Ice Age: The Meltdown
Ice Age: Dawn of the Dinosaurs
Spy Kids
Beginners
Joe Dirt
Friday
Monster's Ball
Jesus' Son
The Amazing Spider-Man
The Shaggy Dog
Epic
The Harvest (La Cosecha)
The Nut Job
Rounders
Robots
Practical Magic
White Oleander
Ride Along
Mrs. Doubtfire
Cold Mountain
The Girl with the Dragon Tattoo
Bolt
Welcome to Mooseport
Mary Poppins
The Sixth Sense
Walking With Dinosaurs
Elektra
Death at a Funeral
